### This code will merge the data transfer obtained after running ietrans.py to a new .h5 file containing the augmented dataset

In [22]:
import h5py
import json
import numpy as np
from tqdm import tqdm

data_path = "../../datasets/IndoorVG/VG-SGG.h5"
new_rels_path = "ietrans.json"
dict_path = "../../datasets/IndoorVG/VG-SGG-dicts.json"

data = h5py.File(data_path, 'r')
new_rels_data = json.load(open(new_rels_path, 'r'))
labels_dict = json.load(open(dict_path, 'r'))

idx_to_labels = labels_dict['idx_to_label']
idx_to_predicates = labels_dict['idx_to_predicate']

relationships = []
predicates = []
im_to_first_rel = np.zeros(len(data['img_to_first_rel']), dtype=np.int32)
im_to_last_rel = np.zeros(len(data['img_to_first_rel']), dtype=np.int32)

rel_idx_counter = 0

print('Before transfer: ', len(data['relationships']))

data_split = data['split_rel'][:]
split_mask = data_split == 0
split_mask &= data['img_to_first_rel'][:] >= 0
image_index = np.where(split_mask)[0]

count_trans = 0
out_count = 0
for i in tqdm(range(len(data['split_rel']))):
    if i not in image_index:
        if data['img_to_first_rel'][i] == -1:
            im_to_first_rel[i] = -1
            im_to_last_rel[i] = -1
        else:
            im_to_first_rel[i] = rel_idx_counter
            range_idx = range(data['img_to_first_rel'][i], data['img_to_last_rel'][i]+1)
            for p in range_idx:
                predicates.append(data['predicates'][p])
                relationships.append(data['relationships'][p])
                rel_idx_counter += 1
                out_count += 1
            im_to_last_rel[i] = rel_idx_counter - 1
    else:
        cur_index = image_index.tolist().index(i)     
        if str(cur_index) not in new_rels_data.keys():
            if data['img_to_first_rel'][i] == -1:
                im_to_first_rel[i] = -1
                im_to_last_rel[i] = -1
            else:
                im_to_first_rel[i] = rel_idx_counter
                range_idx = range(data['img_to_first_rel'][i], data['img_to_last_rel'][i]+1)
                for p in range_idx:
                    predicates.append(data['predicates'][p])
                    relationships.append(data['relationships'][p])
                    rel_idx_counter += 1
                    out_count += 1
                im_to_last_rel[i] = rel_idx_counter - 1
            continue
        new_rels = new_rels_data[str(cur_index)]
        range_idx = range(data['img_to_first_rel'][i], data['img_to_last_rel'][i]+1)
        if len(list(range_idx)) != 0:
            im_to_first_rel[i] = rel_idx_counter
            # internal trans
            for j in range_idx:
                rel = data['relationships'][j]
                if len(new_rels) != 0 and np.any(np.all(np.array(new_rels)[:,0:2] == rel, axis=1)):
                    rel_idx = np.where(np.all(np.array(new_rels)[:,0:2] == rel, axis=1))[0]
                    predicates.append(new_rels[rel_idx[0]][2])
                    new_rels.pop(rel_idx[0])
                else:
                    predicates.append(data['predicates'][j])
            else:
                predicates.append(data['predicates'][j])
            rel_idx_counter += 1
            relationships.append(rel)
        
        # external trans
        for rel in new_rels:
            i_obj_start = data['img_to_first_box'][i]
            num_boxes = data['img_to_last_box'][i] - i_obj_start + 1

            if rel[0] >= num_boxes or rel[1] >= num_boxes:
                print('Error: ', rel[0], rel[1], num_boxes)
                continue

            assert rel[0] < num_boxes, f'{rel[0]} > {num_boxes}'
            assert rel[1] < num_boxes, f'{rel[1]} > {num_boxes}'
            predicates.append(rel[2])
            sub = i_obj_start + rel[0]
            obj = i_obj_start + rel[1]
            relationships.append([sub,obj])
            rel_idx_counter += 1
            count_trans += 1
        if im_to_first_rel[i] == rel_idx_counter:
        # if no qualifying relationship
            im_to_first_rel[i] = -1
            im_to_last_rel[i] = -1
        else:
            im_to_last_rel[i] = rel_idx_counter - 1

print('After transfer: ', len(relationships))